In [1]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


In [4]:
os.chdir("C:/Users/91889/Desktop/Auto tagging")
os.getcwd()

'C:\\Users\\91889\\Desktop\\Auto tagging'

In [5]:
data = pd.read_csv("stack-overflow-data.csv")

In [6]:
data.head()

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python


In [7]:
data['tags'].value_counts()

java             2000
c++              2000
javascript       2000
c#               2000
python           2000
android          2000
html             2000
sql              2000
mysql            2000
jquery           2000
ruby-on-rails    2000
objective-c      2000
php              2000
ios              2000
css              2000
.net             2000
c                2000
iphone           2000
angularjs        2000
asp.net          2000
Name: tags, dtype: int64

In [8]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 32000
Test size: 8000


In [9]:
train_posts = data['post'][:train_size]
train_tags = data['tags'][:train_size]

test_posts = data['post'][train_size:]
test_tags = data['tags'][train_size:]

In [10]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [11]:
tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [12]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [13]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [14]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (32000, 1000)
x_test shape: (8000, 1000)
y_train shape: (32000, 20)
y_test shape: (8000, 20)


In [15]:
batch_size = 32
epochs = 10

In [16]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 28800 samples, validate on 3200 samples
Epoch 1/10
28800/28800 [==============================] - 50s 2ms/step - loss: 0.9819 - accuracy: 0.7184 - val_loss: 0.6135 - val_accuracy: 0.8094
Epoch 2/10
28800/28800 [==============================] - 39s 1ms/step - loss: 0.5523 - accuracy: 0.8232 - val_loss: 0.5996 - val_accuracy: 0.8041
Epoch 3/10
28800/28800 [==============================] - 39s 1ms/step - loss: 0.4562 - accuracy: 0.8509 - val_loss: 0.5975 - val_accuracy: 0.8075
Epoch 4/10
28800/28800 [==============================] - 44s 2ms/step - loss: 0.3898 - accuracy: 0.8698 - val_loss: 0.6239 - val_accuracy: 0.8059
Epoch 5/10
28800/28800 [==============================] - 45s 2ms/step - loss: 0.3227 - accuracy: 0.8917 - val_loss: 0.6465 - val_accuracy: 0.7947
Epoch 6/10
28800/28800 [==============================] - 44s 2ms/step - loss: 0.2698 - accuracy: 0.9093 - val_loss: 0.6706 - val_accuracy: 0.7987
Epoch 7/10
28800/28800 [==============================] - 39s 1ms/ste

In [21]:
model.save("model.h5")

In [19]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

8000/8000 [==============================] - 3s 411us/step
Test score: 0.7997152806520462
Test accuracy: 0.796750009059906


In [20]:
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_posts.iloc[i][:50], "...")
    print('Actual label:' + test_tags.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

jquery( tr_selector ) is removing the style values ...
Actual label:jquery
Predicted label: jquery

web reference in wcf  in wcf if we use webreferenc ...
Actual label:.net
Predicted label: asp.net

fragment transaction custom animation - android  i ...
Actual label:android
Predicted label: android

why can i not use the namespace directive in c++ s ...
Actual label:c++
Predicted label: c++

generating password reset link in c# .net  i must  ...
Actual label:.net
Predicted label: c#

php 5.3 $this versus php 5.4  i am calling a membe ...
Actual label:php
Predicted label: php

undefined method `request_uri  actiondispatch  i w ...
Actual label:ruby-on-rails
Predicted label: ruby-on-rails

my table view is not showing data  i am using a ta ...
Actual label:iphone
Predicted label: iphone

update panel not working correctly   i have added  ...
Actual label:asp.net
Predicted label: asp.net

floating objects in html  is it possible to make a ...
Actual label:html
Predicted label: html

